In [1]:
import numpy as np
import pandas as pd
import math
import itertools
import sys
import torch  
import numpy as np  
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm 
from scipy.stats import beta

import statistics

import import_ipynb
from Environments import Square_Crossroads
from Actor_Critic import Actor_Critic

importing Jupyter notebook from Environments.ipynb
importing Jupyter notebook from Actor_Critic.ipynb


In [2]:
n_times_steps = 3
seed = 10
dist_cars = 0.1

env = Square_Crossroads(seed, dist_cars)
env.reset()
print(env.states)
# action_space = {0:[0,0], 1:[0,0], 2:[0,0], 3:[0,0]}
# total_reward = 0

# state_tensor = torch.zeros([4,2], dtype=torch.float32)


# ac = Actor_Critic(state_tensor, 20, 400)

Environment reset with param
Environment reset with param
{0: [[0, 5], [5, 10], 1, 1], 1: [[5, 0], [0, 5], 1, 1], 2: [[5, 10], [5, 0], 1, 1], 3: [[10, 5], [5, 0], 1, 1]}


In [3]:
def huber_loss(advantage, delta = 1.0):
    
    '''
    Huber loss, given target and prediction
    
    '''
    
    error = abs(advantage)
    
    
    if error.item() > delta:
        loss = 0.5 * delta ** 2 + delta * (error - delta)

    else: 
        loss = 0.5 * error ** 2

    return loss

In [4]:
hyperparameters = {"hidden_size_actor" : 200,
                   "hidden_size_critic" : 400,
                   "lr" : 5e-4,
                   "step_size" : 5,
                   "gamma" : 0.9}

max_episodes = 200
num_steps = 5000
GAMMA = 0.99
delta = 1

In [5]:
def train(env, hyperparameters):
    
    model = Actor_Critic(2, hyperparameters["hidden_size_actor"], hyperparameters["hidden_size_critic"])
    model_optimizer = optim.Adam(model.parameters(), lr = hyperparameters["lr"])
    model_scheduler = optim.lr_scheduler.StepLR(model_optimizer, step_size = hyperparameters["step_size"], gamma = hyperparameters["gamma"])
    
    all_lengths = []
    average_lengths = []
    all_rewards = []
    
    l_actor = []
    l_critic = []
    l_total = []
    all_reward = []
    for episode in range(max_episodes):
        print("Training episode: ", episode)
        
        log_probs = []
        values = []
        rewards = []
        
        env.reset()
        
        states = [v[0] for v in env.states.values()] 
        state_space = torch.tensor([[states[0][0], states[0][1]], 
                                    [states[1][0], states[1][1]],
                                    [states[2][0], states[2][1]],
                                    [states[3][0], states[3][1]]], dtype=torch.float32)

        for steps in tqdm(range(num_steps)):
            
            log_prob = 0
            actions = []
            
            value, policy_dist_alpha, policy_dist_beta, sign = model.forward(state_space)
            
#             print(policy_dist_alpha.requires_grad)
#             print(value.requires_grad)
#             print(policy_dist_beta.requires_grad)
#             print(sign.requires_grad)
            
            
                
            value = value.detach().numpy().mean()
#             print(value)
            for n in range(4):
                tmp_alpha = policy_dist_alpha[n].clone()
                tmp_beta = policy_dist_beta[n].clone()
                
                dist = beta(tmp_alpha.detach().numpy(),tmp_beta.detach().numpy())
                
                action = np.random.beta(tmp_alpha.detach().numpy(), tmp_beta.detach().numpy(), size=2)
#                 print("These are the actions sampled from the beta distribution: ", action)
                log_prob += torch.mean(torch.tensor(dist.logpdf(action)))
                
                # sign: [P(accelerate), P(decelearte)]
#                 print("The softmax scores for accelerate-decelarate: ", sign[n])
                if torch.argmax(sign[n]) == 1:
                    action[0] = -action[0]
#                     print("Decelerate")
#                 else:
#                     print("Accelerate")
#                 print("This is the accelerate/decelarate action: ", action[0])
                actions.append(action)
#             
#             action_1 = np.random.beta(policy_dist_alpha[0].detach().numpy(), policy_dist_beta[0].detach().numpy(), size=2)
#             action_2 = np.random.beta(policy_dist_alpha[1].detach().numpy(), policy_dist_beta[1].detach().numpy(), size=2)
#             action_3 = np.random.beta(policy_dist_alpha[2].detach().numpy(), policy_dist_beta[2].detach().numpy(), size=2)
#             action_4 = np.random.beta(policy_dist_alpha[3].detach().numpy(), policy_dist_beta[3].detach().numpy(), size=2)
            
#             print("The sign of the acceleration is: ",sign)
            
            action_space = torch.tensor(actions, dtype=torch.float32)                
            
            new_state_space, reward, done = env.step(action_space, sign)
            
            rewards.append(reward)
            values.append(value)
            log_probs.append(log_prob.mean())
            
            states = [v[0] for v in env.states.values()] 

            new_state_space = torch.tensor([[states[0][0], states[0][1]], 
                                            [states[1][0], states[1][1]],
                                            [states[2][0], states[2][1]],
                                            [states[3][0], states[3][1]]], dtype=torch.float32)


            state_space = new_state_space

            if done or steps == num_steps-1:
                Qval, _, _, _ = model.forward(state_space)
#                 print("Qval: ",Qval)
                Qval = torch.mean(Qval).detach().numpy()
#                 print("Qval averaged: ",Qval)
                
                Qval = Variable(torch.FloatTensor(Qval), requires_grad = True)
#                 print(Qval.requires_grad)
                
                all_rewards.append(np.sum(rewards))
                all_lengths.append(steps)
                average_lengths.append(np.mean(all_lengths[-10:]))
#                 if episode % 10 == 0:                    
                sys.stdout.write("episode: {}, reward: {}, total length: {}, average length: {} \n".format(episode, np.sum(rewards), steps, average_lengths[-1]))
                print("Episode finished")
                break
        
        # compute Q values
        Qvals = np.zeros_like(values)
#         print("The rewards are: ",rewards)
        for t in reversed(range(len(rewards))):
#             print("I get these rewards: ",rewards)
#             print("I use this Qval: ", Qval)
            Qval = rewards[t] + GAMMA * Qval
#             print("Here is a single Qval: ",Qval)
            Qvals[t] = Qval
        
        #update actor critic
        
#         values = [torch.from_numpy(item).float() for item in values]
#     state_tensor = Variable(state_tensor.float())
        values = Variable(torch.FloatTensor(values), requires_grad=True)
#         print("The values are: ",values)
        Qvals = Variable(torch.FloatTensor(Qvals), requires_grad=True)
#         print("The q values are : ", Qvals)
        
#         print("the non stacked log probs are: ", log_probs)
        log_probs = torch.stack(log_probs)
        
#         print("The log_probs are: ",log_probs)
        
        advantage = Qvals - values
#         print("The advantage is: ",advantage)
        
        avg_advantage = advantage.mean()
#         print("The negative logs are: ", -log_probs)
#         print("The average advantage is: ",avg_advantage)
        avg_log_probs = log_probs.mean()
#         print("The average logs are: ",avg_log_probs)
        
        loss_actor = (avg_log_probs * advantage).mean()
        
        loss_critic = huber_loss(avg_advantage, delta)
        
#         loss_critic = abs(0.5*advantage.mean())
        
#         loss_actor = (-(policy_dist_alpha.sum() + policy_dist_beta.sum() + 1e-5)) * advantage   
        
        print("cricitc loss is: .........",loss_critic)
        print("actor loss is: .........", loss_actor)
        total_loss = loss_actor + loss_critic
        
        print("The total loss is: ",total_loss)
        
        l_actor.append(loss_actor)
        l_critic.append(loss_critic)
        l_total.append(total_loss)
        
        model_optimizer.zero_grad()
        total_loss.backward()
        model_optimizer.step()
#         model_scheduler.step()
        
        all_reward.append(statistics.mean(rewards))

    plt.plot(l_actor)
    plt.legend("Actor loss")    
    plt.show()

    plt.plot(l_critic)
    plt.legend("Critic loss")    
    plt.show()
    
    plt.plot(l_total)
    plt.legend("Total loss")    
    plt.show()

    plt.plot(all_reward)
    plt.legend("Mean reward")    
    plt.show()

## Training

In [1]:
train(env, hyperparameters)